# AgriAutoML Prompt Handler

This notebook implements the natural language processing interface using Vertex AI PaLM API.

In [ ]:
import os
import json
import logging
from typing import Dict, Any, Optional
import vertexai
from vertexai.language_models import TextGenerationModel
from google.oauth2 import service_account
from pydantic import BaseModel, Field
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)

In [ ]:
class PredictionRequest(BaseModel):
    """Structure for parsing prediction requests from natural language."""
    location: str = Field(..., description="Location of the farm or field")
    crop_type: str = Field(..., description="Type of crop being grown")
    planting_date: str = Field(..., description="Date when the crop was planted")
    field_size: float = Field(..., description="Size of the field in hectares")
    additional_context: Optional[Dict[str, Any]] = Field(
        default={},
        description="Any additional context provided in the query"
    )

In [ ]:
# Initialize Vertex AI
vertexai.init(
    project=os.getenv("GCP_PROJECT_ID"),
    location=os.getenv("GCP_REGION"),
    credentials=service_account.Credentials.from_service_account_file(
        os.getenv("GOOGLE_APPLICATION_CREDENTIALS")
    )
)

# Initialize text model
model = TextGenerationModel.from_pretrained("text-bison-32k")

# Set generation parameters
parameters = {
    "temperature": 0.1,
    "max_output_tokens": 256,
    "top_k": 40,
    "top_p": 0.8
}

# Create prompt template
prompt_template = '''Extract relevant information for crop yield prediction from the following query.
Focus on identifying the location, crop type, planting date, and field size.
Return the information in JSON format with the following keys:
- location: string (e.g., "Iowa")
- crop_type: string (e.g., "corn")
- planting_date: string (e.g., "2025-04-15")
- field_size: number (in hectares)
- additional_context: object (any other relevant information)

Query: {query}

JSON Response:'''

In [ ]:
def process_query(query: str) -> Dict[str, Any]:
    """Process a natural language query and return predictions."""
    logging.info(f"Processing query: {query}")
    
    # Extract structured information from query
    prompt = prompt_template.format(query=query)
   
    try:
        # Make prediction using the model
        response = model.predict(
            prompt,
            **parameters
        )
        logging.info(f"Raw API response: {response}")
    except Exception as e:
        logging.error(f"API request failed: {str(e)}")
        raise
    
    try:
        # Extract text from predictions
        prediction_text = response.text
        logging.info(f"Extracted prediction text: {prediction_text}")
        
        request_data = PredictionRequest(**json.loads(prediction_text))
        logging.info(f"Parsed request data: {request_data}")
        return request_data.dict()
    except (json.JSONDecodeError, KeyError, IndexError) as e:
        logging.error(f"Failed to parse model response: {str(e)}")
        raise ValueError(f"Failed to parse model response: {str(e)}")

In [ ]:
# Test the query processing
test_query = "What's the expected yield for my 5 hectare corn field in Iowa that I planted on April 15th?"
result = process_query(test_query)
print(json.dumps(result, indent=2))